# Challenge 2 - ANOVA

In statistics, **Analysis of Variance (ANOVA)** is also used to analyze the differences among group means. The difference between t-test and ANOVA is the former is ued to compare two groups whereas the latter is used to compare three or more groups. [Read more about the difference between t-test and ANOVA](http://b.link/anova24).

From the ANOVA test, you receive two numbers. The first number is called the **F-value** which indicates whether your null-hypothesis can be rejected. The critical F-value that rejects the null-hypothesis varies according to the number of total subjects and the number of subject groups in your experiment. In [this table](http://b.link/eda14) you can find the critical values of the F distribution. **If you are confused by the massive F-distribution table, don't worry. Skip F-value for now and study it at a later time. In this challenge you only need to look at the p-value.**

The p-value is another number yielded by ANOVA which already takes the number of total subjects and the number of experiment groups into consideration. **Typically if your p-value is less than 0.05, you can declare the null-hypothesis is rejected.**

In this challenge, we want to understand whether there are significant differences among various types of pokemons' `Total` value, i.e. Grass vs Poison vs Fire vs Dragon... There are many types of pokemons which makes it a perfect use case for ANOVA.

In [12]:
import scipy.stats as st
import pandas as pd

In [13]:
pokemon_dataset = pd.read_csv("Pokemon.csv")


**To achieve our goal, we use three steps:**

1. **Extract the unique values of the pokemon types.**

1. **Select dataframes for each unique pokemon type.**

1. **Conduct ANOVA analysis across the pokemon types.**

#### First let's obtain the unique values of the pokemon types. These values should be extracted from Type 1 and Type 2 aggregated. Assign the unique values to a variable called `unique_types`.

*Hint: the correct number of unique types is 19 including `NaN`. You can disregard `NaN` in next step.*

In [14]:
type1_values = pokemon_dataset['Type 1'].unique()
type2_values = pokemon_dataset['Type 2'].unique()

unique_types = pd.Series(pd.concat([pd.Series(type1_values), pd.Series(type2_values)]).dropna().unique())

print(unique_types)

0        Grass
1         Fire
2        Water
3          Bug
4       Normal
5       Poison
6     Electric
7       Ground
8        Fairy
9     Fighting
10     Psychic
11        Rock
12       Ghost
13         Ice
14      Dragon
15        Dark
16       Steel
17      Flying
dtype: object


#### Second we will create a list named `pokemon_totals` to contain the `Total` values of each unique type of pokemons.

Why we use a list instead of a dictionary to store the pokemon `Total`? It's because ANOVA only tells us whether there is a significant difference of the group means but does not tell which group(s) are significantly different. Therefore, we don't need know which `Total` belongs to which pokemon type.

*Hints:*

* Loop through `unique_types` and append the selected type's `Total` to `pokemon_groups`.
* Skip the `NaN` value in `unique_types`. `NaN` is a `float` variable which you can find out by using `type()`. The valid pokemon type values are all of the `str` type.
* At the end, the length of your `pokemon_totals` should be 18.

In [15]:
pokemon_totals = []

for pokemon_type in unique_types:
    if isinstance(pokemon_type, str): 
        type_total = pokemon_dataset.loc[(pokemon_dataset['Type 1'] == pokemon_type) | (pokemon_dataset['Type 2'] == pokemon_type), 'Total']
        pokemon_totals.append(type_total.values)

print(len(pokemon_totals))

18


#### Now we run ANOVA test on `pokemon_totals`.

*Hints:*

* To conduct ANOVA, you can use `scipy.stats.f_oneway()`. Here's the [reference](http://b.link/scipy44).

* What if `f_oneway` throws an error because it does not accept `pokemon_totals` as a list? The trick is to add a `*` in front of `pokemon_totals`, e.g. `stats.f_oneway(*pokemon_groups)`. This trick breaks the list and supplies each list item as a parameter for `f_oneway`.

In [17]:
import scipy.stats as stats

f_value, p_value = stats.f_oneway(*pokemon_totals)

print("F-value:", f_value)
print("p-value:", p_value)


F-value: 6.6175382960055344
p-value: 2.6457458815984803e-15


#### Interpret the ANOVA test result. Is the difference significant?

In [ ]:
# Your comment here